## Twitter API V2

- API V1とV2は異なることに注意
- 月に取得可能なtweetの上限は 500,000（Dashboardで確認可能）

### TwitterAPI V2 利用申請

1. Twitter にログイン<br>
アカウントが無ければ、AIITのアカウントでGoogle認証 (OAuth) が簡単
2. 電話番号を登録（登録しないと5で先に進めない）<br>
https://help.twitter.com/ja/managing-your-account/how-to-add-a-phone-number-to-your-account
3. https://developer.twitter.com/ にアクセスし、右上の Sign up をクリック
4. 質問に答えて Let's do this をクリック
5. Developer agreement & policy の Accept Terms & Conditions に同意のチェックを入れて Submit をクリック
6. 認証メールが届くので Confirm your email をクリック
7. アプリケーションの名前を入力し Get keys をクリック
8. Here are your keys. で表示された3つのパスワード (API Key, API Key Secret, Bearer Token) を保存
9. Dashboard をクリックすると、上記3つのパスワードを保存したか再度確認されるので、Yes, I saved them をクリック
10. Dashboard が表示される<br>
https://developer.twitter.com/en/portal/dashboard

参考
- 【2022年最新版】面倒な利用申請が不要に！TwitterAPI v2の申請方法<br>
https://manumaruscript.com/twitterapi-v2-howtouse/

### ユーザー認証の設定

1. Dashboard の左側 Projects & Apps から、利用申請の7で入力したアプリケーションを選択<br>
https://developer.twitter.com/en/portal/dashboard
2. User authentication settings の Set up をクリック
3. 以下の例を参考に入力して Save をクリック
   - App permissions: Read and write
   - Type of App: Web App, Automated App or Bot
   - App info
     - Callback URI / Redirect URL: https://twitter.com/
     - Website URL: https://twitter.com/
4. Here is your OAuth 2.0 Client ID and Client Secret で表示された2つのパスワード (Client ID, Client Secret) を保存
5. Done をクリックすると、上記のパスワードを保存したか再度確認されるので、Yes, I saved it をクリック

### ソフトウェアのインストール

```
conda install -c conda-forge tweepy
```
https://anaconda.org/conda-forge/tweepy

### Tweets の取得

In [ ]:
# OAuth 2.0 認証用トークン
# Bearer_Token='{TwitterAPI V2 利用申請の 8 で保存した Bearer Token}'

# 上記内容を _my_token.py に保存
from _my_token import Bearer_Token

In [ ]:
import tweepy

# アクセス用Clientの作成
client = tweepy.Client(bearer_token=Bearer_Token)

# Tweetの検索
# -is:retweet RTの除外
# lang:ja 日本語
tweets = client.search_recent_tweets(query='AIIT -is:retweet lang:ja', max_results=10)

# 確認
tweets

In [ ]:
import pandas as pd

df = pd.DataFrame(tweets.data)

# 確認
df

### 「いいね」情報の取得

In [ ]:
# Tweetの検索
# - tweet_fields: public_metrics に「いいね」情報 (like_count) が入っている
tweets = client.search_recent_tweets(query='AIIT -is:retweet lang:ja', max_results=10,
                                     tweet_fields = ['author_id', 'public_metrics'])

In [ ]:
df = pd.DataFrame(tweets.data)

# 確認
df

In [ ]:
# public_metrics を展開して DataFrame を作成
df_public_metrics = pd.DataFrame(df.public_metrics.tolist())

# 確認
df_public_metrics

In [ ]:
# 不要な public_metrics の削除
_df = df.drop('public_metrics', axis=1)

# 結合
df = _df.join(df_public_metrics)

# 確認
df

### 「いいね」したユーザ情報取得

In [ ]:
# 「いいね」がついた最初のtweetのIDを取得
tweet_id = df.query('like_count > 0').id.iloc[0]

# 「いいね」したユーザ情報取得
liking_users = client.get_liking_users(id=tweet_id)

# 確認
liking_users.data

### ユーザ情報、フォロー情報、フォロワー情報

In [ ]:
# AIIT公式アカウント
user_id = 371614476

# ユーザ情報の取得
user_info = client.get_user(id=user_id)

# 確認
user_info.data

In [ ]:
# フォローしているユーザ情報の取得
users_following = client.get_users_following(id=user_id)

# 確認
users_following.data

In [ ]:
# ユーザーのフォロワー情報の取得
get_users_followers = client.get_users_followers(id=user_id)

# 確認
get_users_followers.data